# Libraries

In [63]:
import pandas as pd
import numpy as np
import math
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import preprocessing

# Dataset Loading and Pre-Processing


In [64]:
# TODO : Copy Dataframes here
with open('travel_cb.txt') as f:
    lines = f.readlines()

for i in range(len(lines)):
    lines[i]=lines[i].strip()
    
count = 0
dic = {}
inner = {}
for i in range(len(lines)):
    dic[count] = {}
    pos = lines[i].find(":")
    if(pos != -1):
        attr = lines[i][:pos]
        val = lines[i][pos+2:len(lines[i])-1].strip()
        inner[attr] = val
        if(attr == "Hotel"):
            dic[count] = inner
            inner = {}
            count += 1
print(dic[10])

{'JourneyCode': '11', 'HolidayType': 'City', 'Price': '1978', 'NumberOfPersons': '2', 'Region': 'Cairo', 'Transportation': 'Plane', 'Duration': '7', 'Season': 'April', 'Accommodation': 'ThreeStars', 'Hotel': '"Hotel Victoria, Cairo"'}


In [65]:
main=pd.DataFrame.from_dict(dic,orient="index")
N = 3
# Drop last N columns of dataframe
df = main.iloc[: , :-N]
# drop rows with NaN as values
df = df.dropna()
# add feature - weight
df['Weight'] = 1
df

,JourneyCode,HolidayType,Price,NumberOfPersons,Region,Transportation,Duration,Season,Accommodation,Hotel,Weight
0,1,Bathing,2498,2,Egypt,Plane,14,April,TwoStars,"""Hotel White House, Egypt""",1
1,2,Bathing,3066,3,Egypt,Plane,21,May,TwoStars,"""Hotel White House, Egypt""",1
2,3,Active,2778,2,Egypt,Plane,14,June,TwoStars,"""Hotel Geisum, Egypt""",1
3,4,Bathing,3195,3,Egypt,Plane,7,July,ThreeStars,"""Hotel Al Mashrabia, Egypt""",1
4,5,Bathing,3158,2,Egypt,Plane,14,September,ThreeStars,"""Hotel Al Mashrabia, Egypt""",1
...,...,...,...,...,...,...,...,...,...,...,...
1471,1443,Bathing,1518,2,TurkishRiviera,Plane,14,December,ThreeStars,"""Hotel Santana, Turk. Riviera""",1
1473,1465,Recreation,1878,2,Tunisia,Plane,14,November,ThreeStars,"""Hotel Quatre Saisons, Tunisia""",1
1475,1466,Recreation,1765,3,Tunisia,Plane,7,January,ThreeStars,"""Hotel Quatre Saisons, Tunisia""",1
1477,1465,Recreation,1878,2,Tunisia,Plane,14,November,ThreeStars,"""Hotel Quatre Saisons, Tunisia""",1


In [88]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1475 entries, 0 to 1479
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   JourneyCode      1475 non-null   object
 1   HolidayType      1475 non-null   int64 
 2   Price            1475 non-null   object
 3   NumberOfPersons  1475 non-null   object
 4   Region           1475 non-null   int64 
 5   Transportation   1475 non-null   int64 
 6   Duration         1475 non-null   object
 7   Season           1475 non-null   int64 
 8   Accommodation    1475 non-null   int64 
 9   Hotel            1475 non-null   int64 
 10  Weight           1475 non-null   int64 
dtypes: int64(7), object(4)
memory usage: 138.3+ KB


In [67]:
#TODO: Normalize data into numerical values
#creating labelEncoder
le = preprocessing.LabelEncoder()
# Converting string labels into numbers.
# https://stackoverflow.com/questions/42196589/any-way-to-get-mappings-of-a-label-encoder-in-python-pandas
df['Transportation']=le.fit_transform(df['Transportation'])
le.fit(df['Transportation'])
le_transportation_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
df['Transportation']=le.transform(df['Transportation'])
le.fit(df['Season'])
le_season_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
df['Season']=le.transform(df['Season'])
df['Season']

0        0
1        8
2        6
3        5
4       11
        ..
1471     2
1473     9
1475     4
1477     9
1479     4
Name: Season, Length: 1475, dtype: int64

In [68]:
le.fit(df['HolidayType'])
le_holiday_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
df['HolidayType']=le.transform(df['HolidayType'])

le.fit(df['Region'])
le_region_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
df['Region']=le.transform(df['Region'])

le.fit(df['Accommodation'])
le_acc_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
df['Accommodation']=le.transform(df['Accommodation'])

le.fit(df['Hotel'])
le_hotel_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
df['Hotel']=le.transform(df['Hotel'])
df.head()


,JourneyCode,HolidayType,Price,NumberOfPersons,Region,Transportation,Duration,Season,Accommodation,Hotel,Weight
0,1,1,2498,2,27,2,14,0,5,413,1
1,2,1,3066,3,27,2,21,8,5,413,1
2,3,0,2778,2,27,2,14,6,5,260,1
3,4,1,3195,3,27,2,7,5,4,187,1
4,5,1,3158,2,27,2,14,11,4,187,1


## Data Split

In [78]:
# X = data without 'Price'
X = df.iloc[:,0:]
X = X.drop(df.columns[[2]], axis=1)  # df.columns is zero-based pd.Index

X['JourneyCode'] = X['JourneyCode'].astype(str).astype(int)
X['NumberOfPersons'] = X['NumberOfPersons'].astype(str).astype(int)
X['Duration'] = X['Duration'].astype(str).astype(int)
X.info()
X = X.values
X

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1475 entries, 0 to 1479
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   JourneyCode      1475 non-null   int64
 1   HolidayType      1475 non-null   int64
 2   NumberOfPersons  1475 non-null   int64
 3   Region           1475 non-null   int64
 4   Transportation   1475 non-null   int64
 5   Duration         1475 non-null   int64
 6   Season           1475 non-null   int64
 7   Accommodation    1475 non-null   int64
 8   Hotel            1475 non-null   int64
 9   Weight           1475 non-null   int64
dtypes: int64(10)
memory usage: 126.8 KB


array([[   1,    1,    2, ...,    5,  413,    1],
       [   2,    1,    3, ...,    5,  413,    1],
       [   3,    0,    2, ...,    5,  260,    1],
       ...,
       [1466,    5,    3, ...,    4,  354,    1],
       [1465,    5,    2, ...,    4,  354,    1],
       [1466,    5,    3, ...,    4,  354,    1]])

In [75]:
# target label = Price
Y = df['Price']
Y = Y.values

# KNN Algorithm

In [106]:
class KNN:
    def __init__(self, K = 3):
        self.K = K
        
    def euclidean_distance(self,a,b,w = 1): # initial weight = 1
        return np.sqrt(np.sum(w*((a-b)**2)))
    
    def fit(self,X,y):
        self.X_train = X
        self.y_train = y
        
    def predict(self,samples):
        predicted_labels = [self._predict(sample) for sample in samples]
        return np.array(predicted_labels)
    
    def _predict(self,sample):
        # distance between sample and training points
        distances = [self.euclidean_distance(sample,x_train) for x_train in self.X_train]
        # get k-nearest data after sorting
        k_nearest_indices = np.sort(distances)[:self.K]
        # get k-nearest data labels
        k_nearest_labels = [self.y_train[i] for i in k_nearest_indices]
        # arrange frequency to get common class label
        most_common = Counter(k_nearest_labels).most_common(1)
        return most_common[0][0] 

In [107]:
# create a sample data
d = {'JourneyCode':[0],'HolidayType': [0], 'NumberOfPersons' : [2], 'Region':[27], 'Transportation': [0],"Duration":[21],"Season":[6],"Accomodation":[4],"Hotel":[187],"Weight":[1] }
samples = pd.DataFrame(data=d)
samples = samples.values


In [108]:
# check KNN functionality
obj = KNN()
obj.fit(X,Y)

a = obj.predict(samples)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
def generate_query():
    flag=1
    print("Enter the case below:")
    destination=input("Enter your destination region")
    print("Select one out of three questions")
    while(flag):
        ques=["Duration of your stay ", "Number of people for the stay ", "Type of destination (city/countryside) ", "Do you want a more expensive or cheaper accomodation", "Not satisfied with results?", "Need more options?"]        #Not satisfied with results will ask if the customer wants to start over or end the conversation &&& Need more options will display the next three results.
        #add more questions and print only first three questions at a time and display the results each time.
        while(1):
            for i in range(3):
                print(i+") "+ques[i])
            slt=input("Select one: ")
            op=ques[slt-1]
            ques=ques[:slt]+ques[slt+1:]
            if(op=="Duration of your stay"):
                dur=input()
                #insert duration variable into the input vector
                #call for euclidean distance over all samples
                continue
            if(op=="Number of people for the stay"):
                pop=input()
                #same as above
                #calculate the price according to number of people using (total price)/(number of people) if the number of people in the retrived case is not the same
                continue
            if(op=="Type of destination (city/countryside)"):
                pop=input()
                #same as above
                continue
            if(op=="Do you want a more expensive or cheaper accomodation"):
                accprice=input("Enter 1 for better acc and 0 for cheaper acc")
                #go through the list of acc for cheaper or richer options and display 3
                continue
            if(op=="Not satisfied with results?"):
                if(input("Want to start over? 1 for yes 0 for no")==1):
                    break
                else:
                    return
            if(op=="Need more options?"):
                #print next three options
                continue



generate_query()